In [1]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import xmlschema
import networkx as nx
from xml.etree import cElementTree as ElementTree
import matplotlib.pyplot as plt
import re


In [2]:
"""
DISEASE-gene from Orphanet: get only link for VWD for now? (VWD<-VWF)
"""

filepath = "../Data/Orphanet_disease-gene.xml"

# REFERENCE 1
class XmlListConfig(list):
    def __init__(self, aList):
        for element in aList:
            if element:
                # treat like dict
                if len(element) == 1 or element[0].tag != element[1].tag:
                    self.append(XmlDictConfig(element))
                # treat like list
                elif element[0].tag == element[1].tag:
                    self.append(XmlListConfig(element))
            elif element.text:
                text = element.text.strip()
                if text:
                    self.append(text)

class XmlDictConfig(dict):
    '''
    Example usage:

    >>> tree = ElementTree.parse('your_file.xml')
    >>> root = tree.getroot()
    >>> xmldict = XmlDictConfig(root)
    '''
    def __init__(self, parent_element):
        if parent_element.items():
            self.update(dict(parent_element.items()))
        for element in parent_element:
            if element:
                # treat like dict - we assume that if the first two tags
                # in a series are different, then they are all different.
                if len(element) == 1 or element[0].tag != element[1].tag:
                    aDict = XmlDictConfig(element)
                # treat like list - we assume that if the first two tags
                # in a series are the same, then the rest are the same.
                else:
                    # here, we put the list in dictionary; the key is the
                    # tag name the list elements all share in common, and
                    # the value is the list itself 
                    aDict = {element[0].tag: XmlListConfig(element)}
                # if the tag has attributes, add those to the dict
                if element.items():
                    aDict.update(dict(element.items()))
                self.update({element.tag: aDict})
            # this assumes that if you've got an attribute in a tag,
            # you won't be having any text. This may or may not be a 
            # good idea -- time will tell. It works for the way we are
            # currently doing XML configuration files...
#             elif element.items():
#                 self.update({element.tag: dict(element.items())})
            # finally, if there are no child tags and no attributes, extract
            # the text
            else:
                self.update({element.tag: element.text})
                

etree = ET.parse(filepath) #create an ElementTree object 
root = etree.getroot()
xmldict = XmlDictConfig(root)

disease_list = xmldict['DisorderList']['Disorder']

## Make a dataframe with only relevant fields: 
    # OrphaNumber (Orphanet ID)
    # Name (disease name)
    # DisorderGeneAssociationList['Count'] (number of genes associated with the disease)
    ## May need loop for:
    # DisorderGeneAssociationList['DisorderGeneAssociation']['Gene']['Name'] (name of associated gene)
    # DisorderGeneAssociationList['DisorderGeneAssociation']['Gene']['ExternalReferenceList']['ExternalReference'] (Gene IDs and sources)
    # DisorderGeneAssociationList['DisorderGeneAssociation']['Gene']['DisorderGeneAssociationType']['Name']?
    # DisorderGeneAssociationList['DisorderGeneAssociation']['Gene']['DisorderGeneAssociationStatus']['Name'] (whether association is 'Assessed')



In [3]:
disease_list[0]

{'id': '17601',
 'OrphaNumber': '166024',
 'Name': 'Multiple epiphyseal dysplasia, Al-Gazali type',
 'DisorderGeneAssociationList': {'count': '1',
  'DisorderGeneAssociation': {'SourceOfValidation': '22587682[PMID]',
   'Gene': {'id': '20160',
    'OrphaNumber': '268061',
    'Name': 'kinesin family member 7',
    'Symbol': 'KIF7',
    'SynonymList': {'count': '1', 'Synonym': 'JBTS12'},
    'ExternalReferenceList': {'ExternalReference': [{'id': '57240',
       'Source': 'Ensembl',
       'Reference': 'ENSG00000166813'},
      {'id': '51758', 'Source': 'Genatlas', 'Reference': 'KIF7'},
      {'id': '51756', 'Source': 'HGNC', 'Reference': '30497'},
      {'id': '51757', 'Source': 'OMIM', 'Reference': '611254'},
      {'id': '97306', 'Source': 'Reactome', 'Reference': 'Q2M1P5'},
      {'id': '51759', 'Source': 'SwissProt', 'Reference': 'Q2M1P5'}],
     'count': '6'},
    'LocusList': {'count': '1', 'Locus': '\n              '}},
   'DisorderGeneAssociationType': {'id': '17949',
    'Name'

In [3]:
OrphaNumbers = []
Disease_names = []
GeneAssociationCounts = []
Associated_genes = []
Gene_IDs = []
Association_types = []
Association_statuses = []
counter = -1

for disease in disease_list:
    OrphaNumber = int(disease['OrphaNumber'])
    dnames = disease['Name']
    Count = int(disease['DisorderGeneAssociationList']['count'])
    
    associations = disease['DisorderGeneAssociationList']['DisorderGeneAssociation'] # dict if 1 association, list if >1
    genes = []
    gene_symbols = []
    geneIDs = []
    types = []
    statuses = []
    if Count > 1:
        for i, association in enumerate(associations):
            genes.append(associations[i]['Gene']['Name'])
            gene_symbols.append(associations[i]['Gene']['Symbol'])
            geneIDs.append(associations[i]['Gene']['ExternalReferenceList'])
            types.append(associations[i]['DisorderGeneAssociationType']['Name'])
            statuses.append(associations[i]['DisorderGeneAssociationStatus']['Name'])
    else:
        genes.append(associations['Gene']['Name'])
        gene_symbols.append(associations['Gene']['Symbol'])
        geneIDs.append(associations['Gene']['ExternalReferenceList'])
        types.append(associations['DisorderGeneAssociationType']['Name'])
        statuses.append(associations['DisorderGeneAssociationStatus']['Name'])        
    
    # Update column lists
    OrphaNumbers.append(OrphaNumber)
    Disease_names.append(dnames)
    GeneAssociationCounts.append(Count)
    Associated_genes.append(genes)
    Gene_IDs.append(geneIDs)
    Association_types.append(types)
    Association_statuses.append(statuses)
    
    counter += 1

print("Number of terms:" + str(counter))
# colnames = ['OrphaNumber', 'Disease_name', 'GeneAssociationCount', 
#             'Associated_genes', 'Gene_IDs', 'Association_types', 'Association_statuses'] 
coldict = {'OrphaNumber': OrphaNumbers,
          'Disease_name': Disease_names,
          'GeneAssociationCount': GeneAssociationCounts,
          'Associated_genes': Associated_genes,
          'Gene_IDs': Gene_IDs,
          'Association_types': Association_types,
          'Association_statuses': Association_statuses}

disease_df = pd.DataFrame(coldict)
disease_df.to_csv("../Data/Orphanet_disease-gene_df.xlsx", sep = '\t')
disease_df.head(10)


Number of terms:3802


,OrphaNumber,Disease_name,GeneAssociationCount,Associated_genes,Gene_IDs,Association_types,Association_statuses
0,166024,"Multiple epiphyseal dysplasia, Al-Gazali type",1,[kinesin family member 7],"[{'ExternalReference': [{'id': '57240', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]
1,93,Aspartylglucosaminuria,1,[aspartylglucosaminidase],"[{'ExternalReference': [{'id': '126323', 'Sour...",[Disease-causing germline mutation(s) in],[Assessed]
2,166035,Brachydactyly-short stature-retinitis pigmento...,1,[CWC27 spliceosome associated protein homolog],[\n ],[Disease-causing germline mutation(s) in],[Assessed]
3,585,Multiple sulfatase deficiency,1,[sulfatase modifying factor 1],"[{'ExternalReference': [{'id': '56683', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]
4,118,Beta-mannosidosis,1,[mannosidase beta],"[{'ExternalReference': [{'id': '100307', 'Sour...",[Disease-causing germline mutation(s) in],[Assessed]
5,166068,Pontocerebellar hypoplasia type 5,1,[tRNA splicing endonuclease subunit 54],"[{'ExternalReference': [{'id': '58222', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]
6,166063,Pontocerebellar hypoplasia type 4,1,[tRNA splicing endonuclease subunit 54],"[{'ExternalReference': [{'id': '58222', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]
7,166078,Von Willebrand disease type 1,1,[von Willebrand factor],"[{'ExternalReference': [{'id': '60135', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]
8,166073,Pontocerebellar hypoplasia type 6,1,"[arginyl-tRNA synthetase 2, mitochondrial]","[{'ExternalReference': [{'id': '60133', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]
9,166084,Von Willebrand disease type 2A,1,[von Willebrand factor],"[{'ExternalReference': [{'id': '60135', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]


In [4]:
# Disease associations to use: 'Disease-causing' substring
# Other potential associations: 'Part of fusion gene', 'Major susceptibility factor', 'Role in phenotype' substrings

# Keep only entries with causal disease associations:
for i, association in enumerate(disease_df['Association_types']):
    if 'disease-causing' not in str(association).lower():
        disease_df = disease_df.drop(i)
        
print(len(disease_df))
disease_df.head(10)


3415


,OrphaNumber,Disease_name,GeneAssociationCount,Associated_genes,Gene_IDs,Association_types,Association_statuses
0,166024,"Multiple epiphyseal dysplasia, Al-Gazali type",1,[kinesin family member 7],"[{'ExternalReference': [{'id': '57240', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]
1,93,Aspartylglucosaminuria,1,[aspartylglucosaminidase],"[{'ExternalReference': [{'id': '126323', 'Sour...",[Disease-causing germline mutation(s) in],[Assessed]
2,166035,Brachydactyly-short stature-retinitis pigmento...,1,[CWC27 spliceosome associated protein homolog],[\n ],[Disease-causing germline mutation(s) in],[Assessed]
3,585,Multiple sulfatase deficiency,1,[sulfatase modifying factor 1],"[{'ExternalReference': [{'id': '56683', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]
4,118,Beta-mannosidosis,1,[mannosidase beta],"[{'ExternalReference': [{'id': '100307', 'Sour...",[Disease-causing germline mutation(s) in],[Assessed]
5,166068,Pontocerebellar hypoplasia type 5,1,[tRNA splicing endonuclease subunit 54],"[{'ExternalReference': [{'id': '58222', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]
6,166063,Pontocerebellar hypoplasia type 4,1,[tRNA splicing endonuclease subunit 54],"[{'ExternalReference': [{'id': '58222', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]
7,166078,Von Willebrand disease type 1,1,[von Willebrand factor],"[{'ExternalReference': [{'id': '60135', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]
8,166073,Pontocerebellar hypoplasia type 6,1,"[arginyl-tRNA synthetase 2, mitochondrial]","[{'ExternalReference': [{'id': '60133', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]
9,166084,Von Willebrand disease type 2A,1,[von Willebrand factor],"[{'ExternalReference': [{'id': '60135', 'Sourc...",[Disease-causing germline mutation(s) in],[Assessed]


In [5]:
# Build graph! Dataframe columns: source node, source node attributes, target node, target node attributes, edge attributes
NE_cols = ['source_node', 'snode_type', 'external_ref', 
           'target_node', 'tnode_type', 'OrphaNumber', 
           'assoc_type', 'assoc_status']
NE_df = pd.DataFrame(columns = NE_cols)

for i, genelist in enumerate(disease_df['Associated_genes']):
    for j, gene in enumerate(genelist):
        snode = gene
        snode_type = 'gene'
        ext_ref = disease_df.iloc[i]['Gene_IDs'][j]

        tnode = disease_df.iloc[i]['Disease_name']
        tnode_type = 'disease'
        OrphaNum = disease_df.iloc[i]['OrphaNumber']
        
        assoc_type = disease_df.iloc[i]['Association_types'][j]
        status = disease_df.iloc[i]['Association_statuses'][j]
        newrow = [snode, snode_type, ext_ref, tnode, tnode_type, OrphaNum, assoc_type, status]
        NE_df.loc[len(NE_df)] = newrow

NE_df.head(10)
                                                       

,source_node,snode_type,external_ref,target_node,tnode_type,OrphaNumber,assoc_type,assoc_status
0,kinesin family member 7,gene,"{'ExternalReference': [{'id': '57240', 'Source...","Multiple epiphyseal dysplasia, Al-Gazali type",disease,166024,Disease-causing germline mutation(s) in,Assessed
1,aspartylglucosaminidase,gene,"{'ExternalReference': [{'id': '126323', 'Sourc...",Aspartylglucosaminuria,disease,93,Disease-causing germline mutation(s) in,Assessed
2,CWC27 spliceosome associated protein homolog,gene,\n,Brachydactyly-short stature-retinitis pigmento...,disease,166035,Disease-causing germline mutation(s) in,Assessed
3,sulfatase modifying factor 1,gene,"{'ExternalReference': [{'id': '56683', 'Source...",Multiple sulfatase deficiency,disease,585,Disease-causing germline mutation(s) in,Assessed
4,mannosidase beta,gene,"{'ExternalReference': [{'id': '100307', 'Sourc...",Beta-mannosidosis,disease,118,Disease-causing germline mutation(s) in,Assessed
5,tRNA splicing endonuclease subunit 54,gene,"{'ExternalReference': [{'id': '58222', 'Source...",Pontocerebellar hypoplasia type 5,disease,166068,Disease-causing germline mutation(s) in,Assessed
6,tRNA splicing endonuclease subunit 54,gene,"{'ExternalReference': [{'id': '58222', 'Source...",Pontocerebellar hypoplasia type 4,disease,166063,Disease-causing germline mutation(s) in,Assessed
7,von Willebrand factor,gene,"{'ExternalReference': [{'id': '60135', 'Source...",Von Willebrand disease type 1,disease,166078,Disease-causing germline mutation(s) in,Assessed
8,"arginyl-tRNA synthetase 2, mitochondrial",gene,"{'ExternalReference': [{'id': '60133', 'Source...",Pontocerebellar hypoplasia type 6,disease,166073,Disease-causing germline mutation(s) in,Assessed
9,von Willebrand factor,gene,"{'ExternalReference': [{'id': '60135', 'Source...",Von Willebrand disease type 2A,disease,166084,Disease-causing germline mutation(s) in,Assessed


In [6]:
# Create diseases graph object
disease_gene_graph = nx.from_pandas_edgelist(NE_df, source = 'source_node', target = 'target_node', 
                                             edge_attr = ['assoc_type', 'assoc_status'])


In [7]:
# Node attributes: key is node, value is dict of attribute-value pairs
snode_attr = {node: {'node_type': NE_df.iloc[i]['snode_type'], 
                     'external_ref': NE_df.iloc[i]['external_ref']} for i, node in enumerate(NE_df['source_node'])}
tnode_attr = {node: {'node_type': NE_df.iloc[i]['tnode_type'], 
                     'OrphaNumber': NE_df.iloc[i]['OrphaNumber']} for i, node in enumerate(NE_df['target_node'])}

nx.set_node_attributes(disease_gene_graph, snode_attr)
nx.set_node_attributes(disease_gene_graph, tnode_attr)


In [9]:
# Create edgelist file? Test with BioNEV code
filepath = '../Graphs/OrphaNet_disease_gene.edgelist'
with open(filepath, 'wb') as file:
    nx.write_edgelist(disease_gene_graph, file, delimiter = '|')

In [81]:
# Visualize graph (not really helpful at this point)
# nx.draw_networkx(disease_gene_graph, with_labels = False, nodelist = NE_df.source_node.tolist(), node_color = 'r', alpha = 0.5)
# nx.draw_networkx(disease_gene_graph, with_labels = False, nodelist = NE_df.target_node.tolist(), node_color = 'b', alpha = 0.5)
# plt.show()


In [10]:
"""
DRUG-gene target from DrugBank*
*Combine with KEGG? Or just use DB alone

"""
filepath = "../Data/DrugBank full database.xml"
xsdpath = "../Data/drugbank_schema.xsd"

# Decode
xs = xmlschema.XMLSchema(xsdpath)
# etree = ET.parse(filepath) #create an ElementTree object 
# root = etree.getroot()


In [ ]:
# Decode to dictionary
### TAKES A WHILE TO RUN
db_dict = xs.to_dict(filepath)
db_dict.keys()


In [ ]:
# Create list of dictionaries, with each dict containing info for a single drug
drugs = root.getchildren()
drug_list = []
for drug in drugs:
    drugdict = dict()
    for element in drug:
        elementtext = element.text
        elementtag = element.tag.split('}')[1]
        drugdict[elementtag] = elementtext
        
        


In [125]:
# Test
drugs = root.getchildren()
drug0 = drugs[0]

print(drug0.tag.split('}')[1])
print(drug0[-4])
print(drug0[-4].text)
print(drug0[-4].tag)

drug
<Element '{http://www.drugbank.ca}targets' at 0x1306b74f8>

    
{http://www.drugbank.ca}targets


/Users/janiceyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  


In [82]:
"""
DRUG-gene target from KEGG*
*Combine with DrugBank??

"""
filepath = "../Data/KEGG_drug"
with open(filepath, 'r') as file:
    # Separate entries (///), then create dictionary
    entries = file.read().split('\n///\n')
    entrylist = list()
    for i, entry in enumerate(entries):
        entry = entry.split('\n')
        entrydict = dict()
        for row in entry:
            pattern = '\s{3,}'
            row = re.split(pattern, row)
            if row[0] != '':
                key = row[0]
                entrydict[key] = row[1:]
            else:
                if key in entrydict.keys():
                    entrydict[key].append(row[1:])  
        entrylist.append(entrydict)

file.close()

# Only keep drug entries that contain target genes
entrylist = [entry for entry in entrylist if 'TARGET' in list(entry.keys())]


In [80]:
# KEGG API for retrieving target gene references
URL = "http://rest.kegg.jp/find/genes/{}"
entryID = ''

{'ENTRY': ['D00045', 'Drug'],
 'NAME': ['Adenosine (JAN/USP);', ['Adenocard (TN);'], ['Adenoscan (TN)']],
 'FORMULA': ['C10H13N5O4'],
 'EXACT_MASS  267.0968': [],
 'MOL_WEIGHT  267.2413': [],
 'REMARK': ['Same as: C00212',
  ['Therapeutic category: 7990'],
  ['ATC code: C01EB10'],
  ['Chemical structure group: DG00243'],
  ['Product (DG00243): D00045<JP/US> D02300<JP>']],
 'EFFICACY': ['Antiarrhythmic, Cardiac depressant, Diagnostic aid (cardiac stress test), Adenosine receptor agonist'],
 'TARGET': ['ADORA [HSA:134 135 136 140] [KO:K04265 K04266 K04267 K04268]'],
 'INTERACTION  ': [],
 'DBLINKS': ['CAS: 58-61-7',
  ['PubChem: 7847113'],
  ['ChEBI: 16335'],
  ['ChEMBL: CHEMBL477'],
  ['DrugBank: DB00640'],
  ['PDB-CCD: ADN'],
  ['LigandBox: D00045'],
  ['NIKKAJI: J4.501B']],
 'ATOM': ['19',
  ['1', 'N4y N', '27.7735  -15.1762'],
  ['2', 'C8y C', '28.7550  -16.0205'],
  ['3', 'C1y C', '26.4808  -15.5916'],
  ['4', 'C8x C', '28.3947  -13.8350'],
  ['5', 'C8y C', '30.0785  -15.2503'],
  [

In [ ]:
"""
Gene pathways from KEGG

"""
filepath =




In [ ]:
"""
Drug chemical/molecular structure data (ChEMBL?)

"""

In [ ]:
"""
REFERENCES:

Reading from Orphanet disease-gene XML file
>https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary
    >http://code.activestate.com/recipes/410469-xml-as-dictionary/
    
"""